In [30]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow_datasets as tfds
import PIL


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)


Data set

In [4]:
nyu_data, nyu_data_info= tfds.load('nyu_depth_v2', shuffle_files=True,
                 download=False, with_info=True)

train_data_set, validation_data_set = nyu_data['train'], nyu_data['validation']


In [6]:
BUFFER_SIZE = 10
BATCH_SIZE = 8
NUM_EPOCHS = 5

train_data = train_data_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_data = train_data_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Hiperparametros

In [27]:

HEIGHT = 256
WIDTH = 256
CHANNELS = 3
LR = 0.0002
EPOCHS = 30
BATCH_SIZE = 32
KernelSize = (3, 3)

f = [16, 32, 64, 128, 256]


# IA Definition

## Modelo

In [33]:
inputs = Input(shape=(HEIGHT, WIDTH, CHANNELS))

x = inputs

# Encoder
# f[0]
d = Conv2D(f[0], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(d)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[0], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x += d

p1 = MaxPool2D((2, 2), (2, 2))(x)
c1 = x

# f[1]
d = Conv2D(f[1], KernelSize, padding='same', strides=1)(p1)
x = BatchNormalization()(d)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[1], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x += d

p2 = MaxPool2D((2, 2), (2, 2))(x)
c2 = x

# f[2]
d = Conv2D(f[2], KernelSize, padding='same', strides=1)(p2)
x = BatchNormalization()(d)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[2], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x += d

p3 = MaxPool2D((2, 2), (2, 2))(x)
c3 = x

# f[3]
d = Conv2D(f[3], KernelSize, padding='same', strides=1)(p3)
x = BatchNormalization()(d)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[3], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)
x += d

p4 = MaxPool2D((2, 2), (2, 2))(x)
c4 = x

# Bottelneck
x = Conv2D(f[4], KernelSize, padding='same', strides=1)(p4)
x = LeakyReLU(alpha=0.2)(x)
x = Conv2D(f[4], KernelSize, padding='same', strides=1)(x)
x = LeakyReLU(alpha=0.2)(x)

# Decoder
# f[3]
x = UpSampling2D((2,2))(x)
concat = Concatenate()([x, c4])
# concat = Concatenate([x, c4])
x = Conv2D(f[3], KernelSize, padding='same', strides=1)(concat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[3], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

# f[2]
x = UpSampling2D((2,2))(x)
concat = Concatenate()([x, c3])
# concat = Concatenate([x, c3])
x = Conv2D(f[2], KernelSize, padding='same', strides=1)(concat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[2], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

# f[1]
x = UpSampling2D((2,2))(x)
concat = Concatenate()([x, c2])
# concat = Concatenate([x, c2])
x = Conv2D(f[1], KernelSize, padding='same', strides=1)(concat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[1], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

# f[0]
x = UpSampling2D((2,2))(x)
concat = Concatenate()([x, c1])
# concat = Concatenate([x, c2])
x = Conv2D(f[0], KernelSize, padding='same', strides=1)(concat)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(f[0], KernelSize, padding='same', strides=1)(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.2)(x)

x = Conv2D(1,(1,1), padding='same', activation="tanh", name="outputs")(x)

outputs = x

model = Model(inputs,outputs)


In [34]:
optimizer = optimizer = tf.keras.optimizers.Adam(
    learning_rate=LR,
    amsgrad=False,
)

cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction="none"
)

model.compile(optimizer, loss=cross_entropy)

In [35]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_72 (Conv2D)             (None, 256, 256, 16  448         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_64 (BatchN  (None, 256, 256, 16  64         ['conv2d_72[0][0]']              
 ormalization)                  )                                                           